In [1]:
#import all libs
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import lightgbm as lgb

from tqdm import tqdm

import ta
from decimal import ROUND_HALF_UP, Decimal

In [2]:
#import the stock_prices.csv in train_fiels as our target dataset 
path_t = './data/train_files/'
train_price = pd.read_csv(f'{path_t}stock_prices.csv')

In [3]:
train_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332531 entries, 0 to 2332530
Data columns (total 12 columns):
 #   Column            Dtype  
---  ------            -----  
 0   RowId             object 
 1   Date              object 
 2   SecuritiesCode    int64  
 3   Open              float64
 4   High              float64
 5   Low               float64
 6   Close             float64
 7   Volume            int64  
 8   AdjustmentFactor  float64
 9   ExpectedDividend  float64
 10  SupervisionFlag   bool   
 11  Target            float64
dtypes: bool(1), float64(7), int64(2), object(2)
memory usage: 198.0+ MB


In [4]:
#as described in EDA
#AdjustmentFactor: Used to calculate theoretical price/volume when split/reverse-split happens (NOT including dividend/allotment of shares).
#So real close price should calculated by AdjustmentFactor cumulative

In [5]:
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    price.set_index("Date", inplace=True)
    return price

In [6]:
# generate AdjustedClose
train_price=train_price.drop('ExpectedDividend',axis=1).fillna(0)
train_price = adjust_price(train_price)

In [7]:
#now we have high, low, open, close data of everyday. However we need more data of trend singles to decide their performance ranking
#this is featuring step
def get_features_for_predict(price, code):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
        code (int)  : A local code for a listed company
    Returns:
        feature DataFrame (pd.DataFrame)
    """
    close_col = "AdjustedClose"
    feats = price.loc[price["SecuritiesCode"] == code].copy()
    
    # Adds all 42 features
    feats = ta.add_all_ta_features(
        feats, "Open", "High", "Low", close_col, "Volume", fillna=False
    )
    
    # To only add specific features
    # Example: https://github.com/bukosabino/ta/blob/master/examples_to_use/bollinger_band_features_example.py
    # df['bb_bbm'] = indicator_bb.bollinger_mavg()
    # df['bb_bbh'] = indicator_bb.bollinger_hband()
    # df['bb_bbl'] = indicator_bb.bollinger_lband()
    
    # filling data for nan and inf
    feats = feats.fillna(0)
    feats = feats.replace([np.inf, -np.inf], 0)
    # drop AdjustedClose column
    feats = feats.drop([close_col], axis=1)

    return feats

In [17]:
# generate feature
buff = []
# fetch prediction target SecuritiesCodes
codes = sorted(train_price["SecuritiesCode"].unique())
for code in tqdm(codes):
    feat = get_features_for_predict(train_price, code)
    buff.append(feat)
feature = pd.concat(buff)

  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Seri

  0%|▎                                                                                | 8/2000 [00:02<10:27,  3.17it/s]C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Seri

  1%|▋                                                                               | 16/2000 [00:04<09:55,  3.33it/s]C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Seri

  1%|▉                                                                               | 24/2000 [00:07<10:45,  3.06it/s]C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Seri

  2%|█▎                                                                              | 32/2000 [00:10<10:39,  3.08it/s]C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Seri

...

 98%|████████████████████████████████████████████████████████████████████████████▊ | 1968/2000 [10:28<00:10,  3.07it/s]C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Seri

 99%|█████████████████████████████████████████████████████████████████████████████ | 1976/2000 [10:30<00:07,  3.07it/s]C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Seri

 99%|█████████████████████████████████████████████████████████████████████████████▍| 1984/2000 [10:33<00:05,  3.08it/s]C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Seri

100%|█████████████████████████████████████████████████████████████████████████████▋| 1992/2000 [10:35<00:02,  3.07it/s]C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
C:\Users\Administrator\anaconda3\lib\site-packages\ta\trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Seri

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [10:38<00:00,  3.13it/s]


In [9]:
feature.head(10)

,RowId,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,SupervisionFlag,Target,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-04,20170104_1301,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,False,0.000730,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-11.026634,0.000000,0.000000
2017-01-05,20170105_1301,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,False,0.002920,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.145879,-0.145985,-0.145879
2017-01-06,20170106_1301,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,False,-0.001092,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.073046,0.073019,-0.072939
2017-01-10,20170110_1301,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,False,-0.005100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.291971,0.291545,0.218818
2017-01-11,20170111_1301,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,False,-0.003295,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.109170,-0.109230,0.109409
2017-01-12,20170112_1301,1301,2745.0,2747.0,2703.0,2731.0,28700,1.0,False,-0.006613,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.510018,-0.511323,-0.401167
2017-01-13,20170113_1301,1301,2707.0,2730.0,2707.0,2722.0,19400,1.0,False,-0.006657,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.329550,-0.330094,-0.729395
2017-01-16,20170116_1301,1301,2725.0,2725.0,2696.0,2704.0,20100,1.0,False,0.002978,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.661278,-0.663475,-1.385850
2017-01-17,20170117_1301,1301,2702.0,2704.0,2682.0,2686.0,18400,1.0,False,0.001856,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.665680,-0.667906,-2.042305


In [10]:
feature.tail(10)

,RowId,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,SupervisionFlag,Target,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-19,20211119_9997,9997,717.0,722.0,708.0,716.0,267800,1.0,False,-0.012640,...,-3.363391,-2.831364,-0.532027,-0.833834,-2.890354,2.056521,743.739659,0.000000,0.000000,-7.969152
2021-11-22,20211122_9997,9997,708.0,717.0,703.0,712.0,175600,1.0,False,0.004267,...,-3.490111,-2.963113,-0.526998,-3.226345,-2.957553,-0.268792,735.216858,-0.558659,-0.560226,-8.483290
2021-11-24,20211124_9997,9997,714.0,719.0,700.0,703.0,317000,1.0,False,-0.025496,...,-3.650622,-3.100615,-0.550007,-0.482523,-2.462547,1.980024,726.245216,-1.264045,-1.272102,-9.640103
2021-11-25,20211125_9997,9997,701.0,711.0,701.0,706.0,146000,1.0,False,-0.029070,...,-3.706734,-3.221839,-0.484895,-3.898656,-2.749769,-1.148888,720.891859,0.426743,0.425835,-9.254499
2021-11-26,20211126_9997,9997,701.0,702.0,683.0,688.0,265200,1.0,False,-0.001497,...,-3.908763,-3.359224,-0.549540,-2.618040,-2.723423,0.105383,711.884743,-2.549575,-2.582640,-11.568123
2021-11-29,20211129_9997,9997,678.0,679.0,665.0,668.0,320800,1.0,False,0.026987,...,-4.249737,-3.537326,-0.712411,0.192755,-2.140187,2.332943,696.379316,-2.906977,-2.950066,-14.138817
2021-11-30,20211130_9997,9997,670.0,689.0,667.0,667.0,296300,1.0,False,-0.001460,...,-4.488859,-3.727633,-0.761226,1.551434,-1.401863,2.953297,686.217265,-0.149701,-0.149813,-14.267352
2021-12-01,20211201_9997,9997,661.0,688.0,660.0,685.0,339100,1.0,False,0.017544,...,-4.429590,-3.868024,-0.561566,3.844718,-0.352547,4.197265,686.060220,2.698651,2.662879,-11.953728
2021-12-02,20211202_9997,9997,681.0,692.0,680.0,684.0,342900,1.0,False,0.014368,...,-4.346809,-3.963781,-0.383028,5.612814,0.840525,4.772289,685.744007,-0.145985,-0.146092,-12.082262


In [11]:
# all data is ready, next step is spliting test and train 

In [12]:
# split data into TRAIN and TEST
TRAIN_END = "2019-09-30"
# We put a gap in 2019-10-01, because there are alot missing data due to power off in japan.
TEST_START = "2019-10-02"

def get_features_and_label(price, codes, features):
    """
    Args:
        price (pd.DataFrame): loaded price data
        codes  (array) : target codes
        feature (pd.DataFrame): features
    Returns:
        train_X (pd.DataFrame): training data
        train_y (pd.DataFrame): label for train_X
        test_X (pd.DataFrame): test data
        test_y (pd.DataFrame): label for test_X
    """
    # to store splited data
    trains_X, tests_X = [], []
    trains_y, tests_y = [], []

    # generate feature one by one
    for code in tqdm(codes):

        feats = features[features["SecuritiesCode"] == code].dropna()
        labels = get_label(price, code).dropna()

        if feats.shape[0] > 0 and labels.shape[0] > 0:
            # align label and feature indexes
            labels = labels.loc[labels.index.isin(feats.index)]
            feats = feats.loc[feats.index.isin(labels.index)]
            assert (labels.loc[:, "SecuritiesCode"] == feats.loc[:, "SecuritiesCode"]).all()
            labels = labels.loc[:, "label"]

            # split data into TRAIN and TEST
            _train_X = feats[: TRAIN_END]
            _test_X = feats[TEST_START:]

            _train_y = labels[: TRAIN_END]
            _test_y = labels[TEST_START:]
            
            assert len(_train_X) == len(_train_y)
            assert len(_test_X) == len(_test_y)

            # store features
            trains_X.append(_train_X)
            tests_X.append(_test_X)
            # store labels
            trains_y.append(_train_y)
            tests_y.append(_test_y)
            
    # combine features for each codes
    train_X = pd.concat(trains_X)
    test_X = pd.concat(tests_X)
    # combine label for each codes
    train_y = pd.concat(trains_y)
    test_y = pd.concat(tests_y)

    return train_X, train_y, test_X, test_y

In [13]:
def get_label(price, code):
    """ Labelizer
    Args:
        price (pd.DataFrame): dataframe of stock_price.csv
        code (int): Local Code in the universe
    Returns:
        df (pd.DataFrame): label data
    """
    df = price.loc[price["SecuritiesCode"] == code].copy()
    df.loc[:, "label"] = df["Target"]

    return df.loc[:, ["SecuritiesCode", "label"]]

In [14]:
len(codes)

2332531

In [18]:
# generate feature/label
train_X, train_y, test_X, test_y = get_features_and_label(
    train_price, codes, feature
)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:24<00:00, 81.74it/s]


In [19]:
#
feat_cols = [
    "momentum_rsi",
    "trend_macd",
    "trend_kst",
    "trend_ema_fast",
    "volatility_bbm",
    "volatility_atr",
]

In [21]:
# initialize model
pred_model = lgb.LGBMRegressor(random_state=42)
# train
pred_model.fit(train_X[feat_cols].values, train_y)
# prepare result data
result = test_X[["SecuritiesCode"]].copy()
# predict
result.loc[:, "predict"] = pred_model.predict(test_X[feat_cols])
# actual result
result.loc[:, "Target"] = test_y.values

def set_rank(df):
    """
    Args:
        df (pd.DataFrame): including predict column
    Returns:
        df (pd.DataFrame): df with Rank
    """
    # sort records to set Rank
    df = df.sort_values("predict", ascending=False)
    # set Rank starting from 0
    df.loc[:, "Rank"] = np.arange(len(df["predict"]))
    return df

result = result.sort_values(["Date", "predict"], ascending=[True, False])
result = result.groupby("Date").apply(set_rank)

In [22]:
result.head(10)

,SecuritiesCode,predict,Target,Rank
Date,,,,
2019-10-02,8338,0.007581,-0.010638,0
2019-10-02,6580,0.004823,0.067119,1
2019-10-02,8550,0.004269,0.000000,2
2019-10-02,3939,0.004258,0.094077,3
2019-10-02,3491,0.004241,0.044211,4
2019-10-02,6538,0.003643,0.007519,5
2019-10-02,3656,0.003622,-0.077063,6
2019-10-02,4446,0.003426,0.023609,7
2019-10-02,4443,0.003216,0.015264,8
